In [1]:
import pandas as pd 
import numpy as np 


In [2]:
map_df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\sample\sample_data\map_category_code_bank.csv')
data_df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\sample\sample_data\bank_financial_report_2.csv')


In [3]:
map_df.head()

,vi_caption,category_code,en_caption,report_type
0,1.Thu nhập lãi và các khoản thu nhập tương tự,IS_001,Interest income and similar income,income_statement
1,2. Chi phí lãi và các chi phí tương tự,IS_002,Interest expenses and similar charges,income_statement
2,I. Thu nhập lãi thuần,IS_003,Net interest income,income_statement
3,3. Thu nhập từ hoạt động dịch vụ,IS_004,Income from service activities,income_statement
4,4. Chi phí hoạt động dịch vụ,IS_005,Service activity expenses,income_statement


In [4]:
data_df.head()

,stock_code,category_code,year,quarter,data
0,BID,IS_001,2020.0,0.0,100687502
1,BID,IS_001,2021.0,0.0,101007908
2,BID,IS_001,2022.0,0.0,121110719
3,BID,IS_001,2023.0,0.0,152761316
4,BID,IS_010,2020.0,0.0,7992789


## Phân tích cấu trúc tài chính 

In [5]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                           columns='category_code', 
                                           values='data', 
                                           aggfunc='sum')

# Ratio calculation functions
def equity_ratio(equity, total_assets):
    return equity / total_assets if total_assets else None

def long_term_asset_self_financing_ratio(permanent_capital, long_term_assets):
    return permanent_capital / long_term_assets if long_term_assets else None

def fixed_asset_self_financing_ratio(permanent_capital, fixed_assets):
    return permanent_capital / fixed_assets if fixed_assets else None

def general_solvency_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities if total_liabilities else None

def return_on_investment(net_income, total_investment):
    return net_income / total_investment if total_investment else None

def ROIC(NOPAT, invested_capital):
    return NOPAT / invested_capital if invested_capital else None

def return_on_long_term_capital(EBIT, average_long_term_capital):
    return EBIT / average_long_term_capital if average_long_term_capital else None

def basic_earning_power(EBIT, average_total_assets):
    return EBIT / average_total_assets if average_total_assets else None

def debt_to_assets_ratio(total_liabilities, total_assets):
    return total_liabilities / total_assets if total_assets else None

def debt_to_equity_ratio(total_liabilities, equity):
    return total_liabilities / equity if equity else None

def short_term_debt_to_assets_ratio(short_term_liabilities, total_assets):
    return short_term_liabilities / total_assets if total_assets else None

def interest_coverage_ratio(EBIT, interest_expense):
    return EBIT / interest_expense if interest_expense else None

def long_term_debt_to_equity_ratio(long_term_liabilities, equity):
    return long_term_liabilities / equity if equity else None

def short_term_debt_to_equity_ratio(short_term_liabilities, equity):
    return short_term_liabilities / equity if equity else None

# Dictionary to hold functions and corresponding category codes
ratio_functions = {
    'equity_ratio': ['BS_500', 'BS_300'],  # equity, total_assets
    'long_term_asset_self_financing_ratio': ['BS_500', ['BS_220','BS_210','BS_240']],  # permanent_capital (equity ), long_term_assets ( fixed assets, Long-term capital investments, property investments)
    'fixed_asset_self_financing_ratio': ['BS_500', 'BS_220'],  # permanent_capital (equity ), fixed_assets
    'general_solvency_ratio': ['BS_300', 'BS_400'],  #  total_assets, total_liabilities
    'return_on_investment': ['IS_021', 'BS_300'],  # net_income, total_investment (using total assets as example)
    # 'ROIC': ['IS_060', 'BS_200'],  # NOPAT, invested_capital
    # 'return_on_long_term_capital': ['IS_050', 'BS_330'],  # EBIT, long_term_liabilities
    'basic_earning_power': ['IS_017', 'BS_300'],  # EBIT, total_assets
    'debt_to_assets_ratio': ['BS_400', 'BS_300'],  # total_liabilities, total_assets
    'debt_to_equity_ratio': ['BS_400', 'BS_500'],  # total_liabilities, equity
    # 'short_term_debt_to_assets_ratio': ['BS_310', 'BS_270'],  # short_term_liabilities, total_assets
    'interest_coverage_ratio': ['IS_017', 'IS_002'],  # EBIT, interest_expense
    'long_term_debt_to_equity_ratio': [['BS_350','BS_360','BS_372'], 'BS_500'],  # long_term_liabilities(BS_350 + BS_360 + BS_372), equity
    # 'short_term_debt_to_equity_ratio': ['BS_310', 'BS_400']  # short_term_liabilities, equity
}

# Create a DataFrame to store the results
results_1 = []

# Iterate through pivoted data
for index, row in pivot_df.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions.items():
        # Fetch input data from the pivot table
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # For cases like permanent capital (sum of multiple values)
                # Sum multiple values if input_name is a list
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_1.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert results to DataFrame
ratios_df_1 = pd.DataFrame(results_1)

ratios_df_1


,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,equity_ratio,0.076580
1,ABB,2020.0,0.0,long_term_asset_self_financing_ratio,7.475770
2,ABB,2020.0,0.0,fixed_asset_self_financing_ratio,9.324157
3,ABB,2020.0,0.0,general_solvency_ratio,1.082930
4,ABB,2020.0,0.0,return_on_investment,0.009605
...,...,...,...,...,...
2775,VPB,2024.0,2.0,basic_earning_power,0.005187
2776,VPB,2024.0,2.0,debt_to_assets_ratio,0.840273
2777,VPB,2024.0,2.0,debt_to_equity_ratio,5.260664
2778,VPB,2024.0,2.0,interest_coverage_ratio,0.610060


In [ ]:
# # Financing Ratio (Hệ số tài trợ)
# def equity_ratio(equity, total_assets):
#     return equity / total_assets

# #  Long-Term Asset Self-Financing Ratio (Hệ số tự tài trợ tài sản dài hạn)
# # longterm_assets = BS_200
# def long_term_asset_self_financing_ratio(permanent_capital, long_term_assets):
#     return permanent_capital / long_term_assets

# #Fixed Asset Self-Financing Ratio (Hệ số tự tài trợ tài sản cố định - TSCĐ)
# # permanent capital = VCSH(400)+ Nợ dài hạn (330)
# # fixed assets = TSCĐ đã và đang đầu tư(220 + 240/242)
# def fixed_asset_self_financing_ratio(permanent_capital, fixed_assets):
#     return permanent_capital / fixed_assets

# # General Solvency Ratio (Hệ số khả năng thanh toán tổng quát) 
# def general_solvency_ratio(total_liabilities, total_assets):
#     return total_assets / total_liabilities

# # ROI 
# def return_on_investment(net_profit, total_investment):
#     return net_profit / total_investment

# # ROIC (Return on Invested Capital)
# #Net Operating Profit After Taxes (NOPAT) = 60
# def ROIC (NOPAT, invested_capital):
#     return NOPAT / invested_capital

# # Return on Long-Term Capital
# # Earnings Before Interest and Taxes (EBIT) (Lợi nhuận trước thuế và lãi vay) = 50
# def return_on_long_term_capital(EBIT, average_long_term_capital):
#     return EBIT / average_long_term_capital

# # Basic Earning Power (BEP) (Năng lực sinh lời cơ bản)
# def basic_earning_power(EBIT, average_total_assets):
#     return EBIT / average_total_assets

# #Debt-to-Assets Ratio
# def debt_to_assets_ratio(total_liabilities, total_assets):
#     return total_liabilities / total_assets

# # Debt-to-Equity Ratio
# def debt_to_equity_ratio(total_liabilities, equity):
#     return total_liabilities / equity

# # Short-Term Debt-to-Assets Ratio
# def short_term_debt_to_assets_ratio(short_term_liabilities, total_assets):
#     return short_time_liabilities / total_assets

# #Equity-to-Total Assets Ratio:
# def equity_to_total_assets_ratio(equity, total_assets):
#     return equity / total_assets

# #Interest Coverage Ratio:
# def interest_coverage_ratio(EBIT, interest_expense):
#     return EBIT / interest_expense
# # Long-Term Debt-to-Equity Ratio
# #longterm liabilities = BS_330
# def long_term_debt_to_equity_ratio(long_term_liabilities, equity):
#     return long_term_liabilities / equity

# # Short-Term Debt-to-Total Assets Ratio:
# def short_term_debt_to_total_assets_ratio(short_term_liabilities, total_assets):
#     return short_term_liabilities / total_assets



## Phân tích khả năng thanh toán (LIquidity)

In [ ]:
# # Receivables-to-Payables Ratio (Tỷ lệ giữa nợ phải thu và nợ phải trả)
# # accounts_receivable(130+210) / total_liabilities(310+330)
# def receivables_to_payables_ratio(accounts_receivable, total_liabilities):
#     return accounts_receivable / total_liabilities

# #Receivables-to-Total Assets Ratio
# def receivables_to_total_assets_ratio(accounts_receivable, total_assets):
#     return accounts_receivable / total_assets

# #Debt-to-Total Capital Ratio 
# # total_liabilities = BS_300
# # total_capital = BS_440
# def debt_to_total_capital_ratio(total_liabilities, total_capital):
#     return total_liabilities / total_capital

# # the Receivables-to-Sales Ratio (Tỷ lệ giữa nợ phải thu người mua so với tổng số tiền hàng bán ra trong kỳ) 
# # accouts_receivables(131+211) / total_sales (10)
# def receivables_to_sales_ratio(accounts_receivables, total_sales):
#     return accounts_receivables / total_sales

# # Allowance for Doubtful Accounts Ratio (Tỷ lệ giữa dự phòng nợ phải thu khó đòi so với tổng số nợ phải thu người mua)
# # allowance_for_doubtful_accounts(137+219) / accounts_receivables(131+211)
# def allowance_for_doubtful_accounts_ratio(allowance_for_doubtful_accounts, accounts_receivables):
#     return allowance_for_doubtful_accounts / accounts_receivables

# #Asset-to-Debt Ratio
# def asset_to_debt_ratio(total_assets, total_liabilities):
#     return total_assets / total_liabilities

# # Current ratio (Hệ số thanh khoản ngắn hạn)
# # current_assets = BS_100 - BS_151
# def current_ratio(current_assets, current_liabilities):
#     return current_assets / current_liabilities

# # quick ratio (Hệ số thanh khoản nhanh)
# # current assets - inventory (BS_100 - BS_151 - BS_140) / current liabilities
# def quick_ratio(current_assets, inventory, current_liabilities):
#     return (current_assets - inventory) / current_liabilities

# # Cash Ratio (Hệ số thanh khoản tiền mặt)
# # cash_and_cash_equivalents(BS_110) / current_liabilities
# def cash_ratio(cash_and_cash_equivalents, current_liabilities):
#     return cash_and_cash_equivalents / current_liabilities

# # Current Maturity Coverage Ratio= Cash and Cash Equivalents/ Current Maturing Debt 
# # cash_and_cash_equivalents(BS_110) / current_maturing_debt ( TMBCTC)

# # long_term_debt_coverage_ratio = non_current_assets / long_term_debt
# def long_term_debt_coverage_ratio(non_current_assets, non_current_liabilities):
#     return non_current_assets / non_current_liabilities
# # Debt-to-Equity Ratio = Total Liabilities/ Total Equity
# def debt_to_equity_ratio(total_liabilities, total_equity):
#     return total_liabilities / total_equity

# # Long-Term Debt to Equity Capital Ratio
# # non_current_liabilities = BS_330
# def long_term_debt_to_equity_capital_ratio(non_current_liabilities, equity):
#     return non_current_liabilities / equity

# # Time Interest Earned (TIE) = EBIT(IS_050 + IS_023)/ Interest Expense(IS_023)
# def time_interest_earned(EBIT, interest_expense):
#     return EBIT / interest_expense

# # Debt to Tangible Net Worth Ratio = total_liabilities / equity - intangible_assets (BS_227)
# def debt_to_tangible_net_worth_ratio(total_liabilities, equity, intangible_assets):
#     return total_liabilities / (equity - intangible_assets)



 


In [9]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df_2 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                           columns='category_code', 
                                           values='data', 
                                           aggfunc='sum')

# Ratio calculation functions
def receivables_to_payables_ratio(accounts_receivable, total_liabilities):
    return accounts_receivable / total_liabilities if total_liabilities else None

def receivables_to_total_assets_ratio(accounts_receivable, total_assets):
    return accounts_receivable / total_assets if total_assets else None

def debt_to_total_capital_ratio(total_liabilities, total_capital):
    return total_liabilities / total_capital if total_capital else None

def receivables_to_sales_ratio(accounts_receivables, total_sales):
    return accounts_receivables / total_sales if total_sales else None

def allowance_for_loan_customers_ratio(allowance_for_loan_customers, loan_to_customers):
    return allowance_for_loan_customers / loan_to_customers if loan_to_customers else None

def asset_to_debt_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities if total_liabilities else None

def current_ratio(current_assets, current_liabilities):
    return current_assets / current_liabilities if current_liabilities else None

def quick_ratio(current_assets, inventory, current_liabilities):
    return (current_assets - inventory) / current_liabilities if current_liabilities else None

def cash_ratio(cash_and_cash_equivalents, current_liabilities):
    return cash_and_cash_equivalents / current_liabilities if current_liabilities else None

def long_term_debt_coverage_ratio(non_current_assets, non_current_liabilities):
    return non_current_assets / non_current_liabilities if non_current_liabilities else None

def debt_to_equity_ratio(total_liabilities, total_equity):
    return total_liabilities / total_equity if total_equity else None

def long_term_debt_to_equity_capital_ratio(non_current_liabilities, equity):
    return non_current_liabilities / equity if equity else None

def time_interest_earned(EBIT, interest_expense):
    return EBIT / interest_expense if interest_expense else None

def debt_to_tangible_net_worth_ratio(total_liabilities, equity, intangible_assets):
    return total_liabilities / (equity - intangible_assets) if (equity - intangible_assets) else None

ratio_functions = {
    'receivables_to_payables_ratio': [ 'BS_251', 'BS_400'],  # accounts_receivable, total_liabilities
    'receivables_to_total_assets_ratio': [ 'BS_251', 'BS_300'],  # accounts_receivable, total_assets
    'debt_to_total_capital_ratio': ['BS_400', ['BS_400','BS_500']],  # total_liabilities, total_capital
    'receivables_to_sales_ratio': ['BS_251', ['IS_010','IS_001','IS_004']],  # accounts_receivables, total_sales
    'allowance_for_loan_customers_ratio': [ 'BS_169', 'BS_161'],  # allowance_for_loan_customers, loan_to_customers
    'asset_to_debt_ratio': ['BS_300', 'BS_400'],  # total_assets, total_liabilities
    'current_ratio': [[ 'BS_110', 'BS_120', 'BS_131', 'BS_132'],['BS_370','BS_371','BS_373']],  # current_assets_for_liquidity(current_assets = BS_110 + BS_120 + BS_131 + BS_132), current_liabilities(current_liabilities = BS_370 + BS_371 + BS_373)
    # 'quick_ratio': [['BS_100', 'BS_151'], 'BS_140', 'BS_310'],  # current_assets - inventory, current_liabilities
    'cash_ratio': [['BS_110','BS_120'], ['BS_370','BS_371','BS_373']],  # cash_and_cash_equivalents, current_liabilities
    'long_term_debt_coverage_ratio': [['BS_210','BS_220','BS_240'], ['BS_350','BS_360']],  # non_current_assets, non_current_liabilities
    'debt_to_equity_ratio': ['BS_400', 'BS_500'],  # total_liabilities, total_equity
    'long_term_debt_to_equity_capital_ratio': [['BS_350','BS_360'], 'BS_500'],  # long_term_liabilities, equity
    'time_interest_earned': ['IS_017', 'IS_002'],  # EBIT, interest_expense
    'debt_to_tangible_net_worth_ratio': ['BS_400', 'BS_500', 'BS_227'],  # total_liabilities, equity, intangible_assets
}

# Create a DataFrame to store the results
results_2 = []

# Iterate through pivoted data
for index, row in pivot_df_2.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions.items():
        input_values = []
            # For all other ratios, sum values if needed or fetch directly
        for input_name in inputs:
            if isinstance(input_name, list):
                 value_sum = sum([row[i] for i in input_name if i in row.index])
                 input_values.append(value_sum)
            else:
                 input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_2.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert results to DataFrame
ratios_df_2 = pd.DataFrame(results_2)

# Display or export the DataFrame
ratios_df_2


,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,receivables_to_payables_ratio,0.016266
1,ABB,2020.0,0.0,receivables_to_total_assets_ratio,0.015021
2,ABB,2020.0,0.0,debt_to_total_capital_ratio,0.923420
3,ABB,2020.0,0.0,receivables_to_sales_ratio,0.240871
4,ABB,2020.0,0.0,allowance_for_loan_customers_ratio,-0.011141
...,...,...,...,...,...
3609,VPB,2024.0,2.0,long_term_debt_coverage_ratio,0.039022
3610,VPB,2024.0,2.0,debt_to_equity_ratio,5.260664
3611,VPB,2024.0,2.0,long_term_debt_to_equity_capital_ratio,0.392673
3612,VPB,2024.0,2.0,time_interest_earned,0.610060


## Phân tích rủi ro tài chính (Financial risk)

In [ ]:
# # financial_leverage = total_liabilities(BS_300) / total_lia_and_equity (BS_440)
# # 0< financial_leverage < 1 : Close to 0 is better
# def financial_leverage(total_liabilities, total_lia_and_equity):
#     return total_liabilities / total_lia_and_equity

# # Allowance for Doubtful Accounts to Total Assets Ratio = allowance_for_doubtful_accounts(BS_137+BS_219)/ total_assets (BS_270)
# def allowance_for_doubtful_accounts_to_total_assets_ratio(allowance_for_doubtful_accounts, total_assets):
#     return allowance_for_doubtful_accounts / total_assets

# # Hệ số tài trợ thường xuyên (Permanent Financing Ratio) = permanent_capital(BS_400+BS_330) / total_lia_and_equity (BS_440)
# def permanent_financing_ratio(permanent_capital, total_lia_and_equity):
#     return permanent_capital / total_lia_and_equity





In [10]:
import pandas as pd

# Create a new pivot table for the new ratios
pivot_df_3 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')


def financial_leverage(total_liabilities, total_lia_and_equity):
    return total_liabilities / total_lia_and_equity if total_lia_and_equity else None


def allowance_for_doubtful_accounts_to_total_assets_ratio(allowance_for_doubtful_accounts, total_assets):
    return allowance_for_doubtful_accounts / total_assets if total_assets else None


def permanent_financing_ratio(permanent_capital, total_lia_and_equity):
    return permanent_capital / total_lia_and_equity if total_lia_and_equity else None

new_ratio_functions = {
    'financial_leverage': ['BS_400', 'BS_800'],  # total_liabilities (BS_300), total_lia_and_equity 
    'allowance_for_doubtful_accounts_to_total_assets_ratio': ['BS_169', 'BS_300'],  # allowance_for_doubtful_accounts , total_assets 
    # 'permanent_financing_ratio': [, 'BS_440'],  # permanent_capital (BS_400 + BS_330), total_lia_and_equity 
}

# Create a DataFrame to store the results of the new ratios
results_3 = []

# Iterate through the pivot table to calculate the new ratios
for index, row in pivot_df_3.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in new_ratio_functions.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like permanent capital or allowance for doubtful accounts
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_3.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_3 = pd.DataFrame(results_3)

# Display or export the DataFrame
ratios_df_3


,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,financial_leverage,0.923420
1,ABB,2020.0,0.0,allowance_for_doubtful_accounts_to_total_asset...,-0.006059
2,ABB,2021.0,0.0,financial_leverage,0.903015
3,ABB,2021.0,0.0,allowance_for_doubtful_accounts_to_total_asset...,-0.006705
4,ABB,2021.0,3.0,financial_leverage,0.912322
...,...,...,...,...,...
551,VPB,2023.0,3.0,allowance_for_doubtful_accounts_to_total_asset...,-0.016364
552,VPB,2024.0,1.0,financial_leverage,0.826188
553,VPB,2024.0,1.0,allowance_for_doubtful_accounts_to_total_asset...,-0.018307
554,VPB,2024.0,2.0,financial_leverage,0.840273


## phân tích kết quả kinh doanh 

In [ ]:
# # Doanh thu thuần về bán hàng và cung cấp dịch vụ ,IS_010
# # Doanh thu hoạt động tài chính,IS_021,Financial income
# # doanh thu thuần = IS_010 + IS_021

# # Financial Income to Net Revenue Ratio = Financial Income(IS_021)/ Net Revenue
# def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
#     return financial_income / net_revenue



In [15]:
import pandas as pd

# Create a new pivot table for the Financial Income to Net Revenue Ratio
pivot_df_4 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Financial Income to Net Revenue Ratio = Financial Income(IS_021) / Net Revenue
def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
    return financial_income / net_revenue if net_revenue else None

# Dictionary to hold the category codes for the new ratio
new_ratio_function = {
    'financial_income_to_net_revenue_ratio': [['IS_004','IS_007','IS_008','IS_009','IS_010'], ['IS_003','IS_004','IS_007','IS_008','IS_009','IS_010']]  # financial_income (IS_004 + IS_007 + IS_008 + IS_009 + IS_010), net_revenue (IS_010)
}

# Create a DataFrame to store the results of the new ratio
results_4 = []

# Iterate through the pivot table to calculate the new ratio
for index, row in pivot_df_4.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in new_ratio_function.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like permanent capital or allowance for doubtful accounts
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_4.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_4 = pd.DataFrame(results_4)

# Display or export the DataFrame
ratios_df_4


,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,financial_income_to_net_revenue_ratio,0.407535
1,ABB,2021.0,0.0,financial_income_to_net_revenue_ratio,0.358191
2,ABB,2021.0,3.0,financial_income_to_net_revenue_ratio,0.200785
3,ABB,2021.0,4.0,financial_income_to_net_revenue_ratio,0.402170
4,ABB,2022.0,0.0,financial_income_to_net_revenue_ratio,0.161032
...,...,...,...,...,...
273,VPB,2023.0,1.0,financial_income_to_net_revenue_ratio,0.339538
274,VPB,2023.0,2.0,financial_income_to_net_revenue_ratio,0.397747
275,VPB,2023.0,3.0,financial_income_to_net_revenue_ratio,0.329223
276,VPB,2024.0,1.0,financial_income_to_net_revenue_ratio,0.286033


## phân tích khả năng sinh lời ( profitability )

In [ ]:
# # ROA : Return on Assets = net_income(IS_060) / total_assets
# def return_on_assets(net_income, total_assets):
#     return net_income / total_assets

# #Return on fixed assets = net_income(IS_060) / average_fixed_assets (BS_220)
# def return_on_fixed_assets(net_income, average_fixed_assets):
#     return net_income / average_fixed_assets

# # #RNOA= Net Operating Profit After Tax (NOPAT)/Average Net Operating Assets
# # def RNOA(NOPAT, average_net_operating_assets):
# #     return NOPAT / average_net_operating_assets

# # return on long-term operating assets = net_income(IS_060) / average_long_term_operating_assets
# def return_on_long_term_operating_assets(net_income, average_long_term_operating_assets):
#     return net_income / average_long_term_operating_assets

# # SỨC SINH LỢI CƠ BẢN CỦA TÀI SẢN : BEPR (Basic Earning Power Ratio) = EBIT(IS_050 +IS_023) / total_assets
# def BEPR(EBIT, total_assets):
#     return EBIT / total_assets

# #ROE = Net Income/  Equity
# def ROE(net_income, equity):
#     return net_income / equity

# # Return on Common Equity (sức sinh lời của vốn cổ phần thường) = Net Income (IS_060) - (Preferred Dividends-CF-036)/ Average Common Equity
# def return_on_common_equity(net_income, preferred_dividends, average_common_equity):
#     return net_income - preferred_dividends / average_common_equity

# # Profitability of Cost of Goods Sold (COGS) = net income from operating(IS_030)/ COGS (IS_011)
# def profitability_of_cost_of_goods_sold(net_income_from_operating, COGS):
#     return net_income_from_operating / COGS

# # Hệ số chênh lệch giá (Price Spread Ratio): gross_profit(IS_020) / COGS(IS_011)
# def price_spread_ratio(gross_profit, COGS):
#     return gross_profit / COGS

# # Sức sinh lợi của chi phí kinh doanh (Profitability of Operating Expenses): net_income_from_operating(IS_030) / total_operating_expenses(IS_025+IS_026+IS_011)
# def profitability_of_operating_expenses(net_income_from_operating, total_operating_expenses):
#     return net_income_from_operating / total_operating_expenses

# # ROS = Net Income/ Net Sales (IS_010)
# def ROS(net_income, net_sales):
#     return net_income / net_sales

# # adjusted ROS = net_income(IS_060) - cổ tức ưu đãi/ net_sales(IS_010) 

# # Operating Profit Margin (OPM) = Net profit from operating activities (IS_030)/ Net Sales
# def operating_profit_margin(net_profit_from_operating, net_sales):
#     return net_profit_from_operating / net_sales

# # Gross Profit Margin (GPM) = Gross Profit(IS_020)/ Net Sales
# def gross_profit_margin(gross_profit, net_sales):
#     return gross_profit / net_sales

# #net operating profit margin = lợi nhuận sau thuế từ hđkd(net operating profit after tax, NOPAT)/ net sales(IS_010)
# # NOPAT = lợi nhuận trc thuế từ hđkd x (1 - thuế suất)
# def net_operating_profit_margin(NOPAT, net_sales):
#     return NOPAT / net_sales

# # price to book ratio = giá cổ phiếu / giá trị sổ sách ( market price per share/book value per share)
# # price to earnings ratio = giá cổ phiếu / lợi nhuận trên cổ phiếu
# #Dividend Payout Ratio =  Dividend per Share/ Earnings per Share
# # Dividend Yield = Dividend per Share/ Market Price per Share





In [ ]:
import pandas as pd

# Create a new pivot table for the ratios in pivot_df_5
pivot_df_5 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Ratio calculation functions
def return_on_assets(net_income, total_assets):
    return net_income / total_assets if total_assets else None

def return_on_fixed_assets(net_income, average_fixed_assets):
    return net_income / average_fixed_assets if average_fixed_assets else None

def return_on_long_term_operating_assets(net_income, average_long_term_operating_assets):
    return net_income / average_long_term_operating_assets if average_long_term_operating_assets else None

def Basic_Earning_Power_Ratio(EBIT, total_assets):
    return EBIT / total_assets if total_assets else None

def Return_on_equity(net_income, equity):
    return net_income / equity if equity else None

# def return_on_common_equity(net_income, preferred_dividends, average_common_equity):
#     return (net_income - preferred_dividends) / average_common_equity if average_common_equity else None

def profitability_of_cost_of_goods_sold(net_income_from_operating, COGS):
    return net_income_from_operating / COGS if COGS else None

def price_spread_ratio(gross_profit, COGS):
    return gross_profit / COGS if COGS else None

def profitability_of_operating_expenses(net_income_from_operating, total_operating_expenses):
    return net_income_from_operating / total_operating_expenses if total_operating_expenses else None

def Return_on_sales(net_income, net_sales):
    return net_income / net_sales if net_sales else None

def operating_profit_margin(profit_from_operating,operating_expense, net_sales):
    return (profit_from_operating - operating_expense)/ net_sales if net_sales else None

def gross_profit_margin(gross_profit, net_sales):
    return gross_profit / net_sales if net_sales else None

# Dictionary to hold functions and corresponding category codes
ratio_functions_5 = {
    'return_on_assets': ['IS_021', 'BS_300'],  # net_income , total_assets 
    'return_on_fixed_assets': ['IS_021', 'BS_220'],  # net_income , average_fixed_assets 
    'return_on_long_term_operating_assets': ['IS_021', ['BS_210','BS_220','BS_240']],  # net_income , average_long_term_operating_assets 
    'Basic_Earning_Power_Ratio': ['IS_017', 'BS_300'],  # EBIT , total_assets 
    'Return_on_equity': ['IS_021', 'BS_500'],  # net_income , equity 
    # 'return_on_common_equity': ['IS_060', 'CF_036', 'BS_400'],  # net_income (IS_060), preferred_dividends (CF_036), average_common_equity (BS_400)
    # 'profitability_of_cost_of_goods_sold': ['IS_030', 'IS_011'],  # net_income_from_operating (IS_030), COGS (IS_011)
    # 'price_spread_ratio': ['IS_020', 'IS_011'],  # gross_profit (IS_020), COGS (IS_011)
    'profitability_of_operating_expenses': ['IS_017', 'IS_014'],  # net_income_from_operating , total_operating_expenses 
    'Return_on_sales': ['IS_021', ['IS_010','IS_003','IS_004']],  # net_income , net_sales 
    'operating_profit_margin': [['IS_010','IS_003','IS_004'],'IS_014', ['IS_010','IS_003','IS_004']],  # NOPAT, net_sales 
    # 'gross_profit_margin': ['IS_020', ['IS_010','IS_003','IS_004','IS_007','IS_008','IS_009']],  # gross_profit , net_sales 
}

# Create a DataFrame to store the results of the new ratios
results_5 = []

# Iterate through the pivot table to calculate the new ratios
for index, row in pivot_df_5.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions_5.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like EBIT or total operating expenses
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_5.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_5 = pd.DataFrame(results_5)

# Display or export the DataFrame
ratios_df_5


,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,return_on_assets,0.009605
1,ABB,2020.0,0.0,return_on_fixed_assets,1.169431
2,ABB,2020.0,0.0,return_on_long_term_operating_assets,0.937607
3,ABB,2020.0,0.0,Basic_Earning_Power_Ratio,0.012058
4,ABB,2020.0,0.0,Return_on_equity,0.125420
...,...,...,...,...,...
2219,VPB,2024.0,2.0,Basic_Earning_Power_Ratio,0.005187
2220,VPB,2024.0,2.0,Return_on_equity,0.026311
2221,VPB,2024.0,2.0,profitability_of_operating_expenses,1.345930
2222,VPB,2024.0,2.0,Return_on_sales,0.199907


## phân tích cashflow


In [ ]:
# # Earnings Before Interest, Taxes, Depreciation and Amortization (EBITDA) = EBIT(IS_050) + Depreciation and Amortization (CF_002)
# def EBITDA(EBIT, depreciation_and_amortization):
#     return EBIT + depreciation_and_amortization
# # Cash Flow Adequacy Ratio = Cash Flow from Operations/ Capital_Expenditures+Debt_Repayments+Dividends_Paid
# # Free Cash Flow (FCF)=Operating_Net_Cash_Flow (CF_020) − Capital_Expenditures_(CAPEX) ( CF_021+CF_023) - Chi trả cổ tức (CF_036)
# def free_cash_flow(operating_net_cash_flow, capital_expenditures, dividends_paid):
#     return operating_net_cash_flow - capital_expenditures - dividends_paid

# # free cash flow to the operating cash flow ratio = free_cash_flow / operating_net_cash_flow (CF_020)
# def free_cash_flow_to_operating_cash_flow_ratio(free_cash_flow, operating_net_cash_flow):
#     return free_cash_flow / operating_net_cash_flow

# # cash debt coverage ratio = operating_net_cash_flow(CF_020)/ avg_total_liabilities
# def cash_debt_coverage_ratio(operating_net_cash_flow, avg_total_liabilities):
#     return operating_net_cash_flow / avg_total_liabilities

# #Hệ số đảm bảo khả năng chi trả lãi vay: cash interest covarage(CIC)= operating_net_cash_flow(CF_020) + interest_expense(IS_023) / interest_expense(IS_023)
# def cash_interest_coverage(operating_net_cash_flow, interest_expense):
#     return operating_net_cash_flow + interest_expense / interest_expense

# # Hệ số khả năng chi trả cổ tức : Dividend payment ratio = perating_net_cash_flow(CF_020)/ dividends_payables (TMBCTC)

# # cash return on assets = operating_net_cash_flow(CF_020) / avg_total_assets
# def cash_return_on_assets(operating_net_cash_flow, avg_total_assets):
#     return operating_net_cash_flow / avg_total_assets

# #cash return on fixed assets = operating_net_cash_flow(CF_020) / avg_fixed_assets
# def cash_return_on_fixed_assets(operating_net_cash_flow, avg_fixed_assets):
#     return operating_net_cash_flow / avg_fixed_assets

# # CFO to total equity = operating_net_cash_flow(CF_020) / avg_total_equity
# def CFO_to_total_equity(operating_net_cash_flow, avg_total_equity):
#     return operating_net_cash_flow / avg_total_equity

# # cash flow return on investment = Net_Cash_Flow_before_Tax_and_Interest_Payments(CF_050 +CF_014 + CF_015) / VỐN SỬ DỤNG BÌNH QUÂN (BS_400+BS_330)

# # Vốn cổ phần phổ thông(Common Equity) = BS_411 - BS_415
# # Cash return on shareholders = operating_net_cash_flow(CF_020)- cổ tức ưu đãi/ số cổ phiếu phổ thông bình quân trong kỳ
# # cas flow from sales to sales = operating_net_cash_flow(CF_020)- CF_036/ net_sales(IS_010)
# def cash_flow_from_sales_to_sales(operating_net_cash_flow, net_sales):
#     return operating_net_cash_flow / net_sales

# # cash flow margin = operating_net_cash_flow(CF_020) / tổng doanh thu thuần (IS_010+ IS_021)
# def cash_flow_margin(operating_net_cash_flow, total_revenue):
#     return operating_net_cash_flow / total_revenue

# # Earning quality ratio = operating_net_cash_flow(CF_020) / net_income(IS_060)
# def earning_quality_ratio(operating_net_cash_flow, net_income):
#     return operating_net_cash_flow / net_income
# # free cash flows to the firm (FCFF) = EBIT (IS_050 +IS_023). (1 - tax rate) + Depreciation  - Capital Expenditures - Change in Net Working Capital 


In [ ]:
import pandas as pd

# Create a new pivot table for the cash flow ratios in pivot_df_7
pivot_df_6 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Function to get the value of quarter 0 from the previous year for a given stock_code
def get_previous_year_q0_value(stock_code, year, category_code):
    try:
        return pivot_df_6.loc[(stock_code, year - 1, 0), category_code]
    except KeyError:
        return None

# Ratio calculation functions
def EBITDA(EBIT, depreciation_and_amortization):
    return EBIT + depreciation_and_amortization

def free_cash_flow(operating_net_cash_flow, capital_expenditures, dividends_paid):
    return operating_net_cash_flow - capital_expenditures - dividends_paid

def free_cash_flow_to_operating_cash_flow_ratio(free_cash_flow, operating_net_cash_flow):
    return free_cash_flow / operating_net_cash_flow if operating_net_cash_flow else None

def cash_debt_coverage_ratio(operating_net_cash_flow, avg_total_liabilities):
    return operating_net_cash_flow / avg_total_liabilities if avg_total_liabilities else None

def cash_interest_coverage(operating_net_cash_flow, interest_expense):
    return (operating_net_cash_flow + interest_expense) / interest_expense if interest_expense else None

def cash_return_on_assets(operating_net_cash_flow, avg_total_assets):
    return operating_net_cash_flow / avg_total_assets if avg_total_assets else None

def cash_return_on_fixed_assets(operating_net_cash_flow, avg_fixed_assets):
    return operating_net_cash_flow / avg_fixed_assets if avg_fixed_assets else None

def CFO_to_total_equity(operating_net_cash_flow, avg_total_equity):
    return operating_net_cash_flow / avg_total_equity if avg_total_equity else None

def cash_flow_from_sales_to_sales(operating_net_cash_flow, net_sales):
    return operating_net_cash_flow / net_sales if net_sales else None

def cash_flow_margin(operating_net_cash_flow, total_revenue):
    return operating_net_cash_flow / total_revenue if total_revenue else None

def earning_quality_ratio(operating_net_cash_flow, net_income):
    return operating_net_cash_flow / net_income if net_income else None
def net_interest_margin(net_interest_income, avg_earning_assets):
    return net_interest_income / avg_earning_assets if avg_earning_assets else None
# Dictionary to map ratios to category codes (with average calculations for avg fields)
cash_flow_ratio_functions_6 = {
    # 'EBITDA': ['IS_050', 'CF_002'],  # EBIT (IS_050), depreciation_and_amortization (CF_002)
    'free_cash_flow': ['CF_024', 'CF_025', 'CF_038'],  # operating_net_cash_flow , capital_expenditures , dividends_paid 
    'free_cash_flow_to_operating_cash_flow_ratio': ['free_cash_flow', 'CF_024'],  # free_cash_flow, operating_net_cash_flow 
    'cash_debt_coverage_ratio': ['CF_024', 'BS_400'],  # operating_net_cash_flow , avg_total_liabilities 
    'cash_interest_coverage': ['CF_024', 'IS_002'],  # operating_net_cash_flow , interest_expense
    'cash_return_on_assets': ['CF_024', 'BS_300'],  # operating_net_cash_flow , avg_total_assets 
    'cash_return_on_fixed_assets': ['CF_024', 'BS_220'],  # operating_net_cash_flow , avg_fixed_assets 
    'CFO_to_total_equity': ['CF_024', 'BS_500'],  # operating_net_cash_flow , avg_total_equity 
    'cash_flow_from_sales_to_sales': ['CF_024', ['IS_010','IS_003','IS_004']],  # operating_net_cash_flow , net_sales 
    'cash_flow_margin': ['CF_020', ['IS_010','IS_003','IS_004','IS_007','IS_008','IS_009']],  # operating_net_cash_flow , total_revenue 
    'earning_quality_ratio': ['CF_024', 'IS_021'],  # operating_net_cash_flow , net_income 
    'net_interest_margin': ['IS_003', ['BS_161','BS_130','BS_170']],  # net_interest_income , avg_earning_assets
}

# Create a DataFrame to store the results of the new cash flow ratios
cash_flow_results_6 = []

# Iterate through the pivot table to calculate the cash flow ratios
for index, row in pivot_df_6.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in cash_flow_ratio_functions_6.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like capital_expenditures or total_revenue
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                if input_name in ['BS_400', 'BS_300', 'BS_220', 'BS_500']:  
                    prev_q0_value = get_previous_year_q0_value(stock_code, year, input_name)
                    current_value = row[input_name] if input_name in row.index else None
                    if current_value is not None and prev_q0_value is not None:
                        avg_value = (current_value + prev_q0_value) / 2
                        input_values.append(avg_value)
                    else:
                        input_values.append(None)
                else:
                    input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            cash_flow_results_6.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_6 = pd.DataFrame(cash_flow_results_6)

# Display or export the DataFrame
ratios_df_6


,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,free_cash_flow,4.380337e+06
1,ABB,2020.0,0.0,cash_interest_coverage,1.996641e+00
2,ABB,2020.0,0.0,cash_flow_from_sales_to_sales,1.452045e+00
3,ABB,2020.0,0.0,cash_flow_margin,-5.577445e-02
4,ABB,2020.0,0.0,earning_quality_ratio,3.837158e+00
...,...,...,...,...,...
2579,VPB,2024.0,2.0,CFO_to_total_equity,1.190200e-01
2580,VPB,2024.0,2.0,cash_flow_from_sales_to_sales,9.099568e-01
2581,VPB,2024.0,2.0,cash_flow_margin,-3.463367e-04
2582,VPB,2024.0,2.0,earning_quality_ratio,4.551894e+00


In [43]:
# Concatenate all the DataFrames
ratios_df = pd.concat([ratios_df_1, ratios_df_2, ratios_df_3, ratios_df_4, ratios_df_5, ratios_df_6], ignore_index=True)

# Display the concatenated DataFrame
ratios_df

,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,equity_ratio,0.076580
1,ABB,2020.0,0.0,long_term_asset_self_financing_ratio,7.475770
2,ABB,2020.0,0.0,fixed_asset_self_financing_ratio,9.324157
3,ABB,2020.0,0.0,general_solvency_ratio,1.082930
4,ABB,2020.0,0.0,return_on_investment,0.009605
...,...,...,...,...,...
12031,VPB,2024.0,2.0,CFO_to_total_equity,0.119020
12032,VPB,2024.0,2.0,cash_flow_from_sales_to_sales,0.909957
12033,VPB,2024.0,2.0,cash_flow_margin,-0.000346
12034,VPB,2024.0,2.0,earning_quality_ratio,4.551894


In [28]:
ratios_df

,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,equity_ratio,0.076580
1,ABB,2020.0,0.0,long_term_asset_self_financing_ratio,7.475770
2,ABB,2020.0,0.0,fixed_asset_self_financing_ratio,9.324157
3,ABB,2020.0,0.0,general_solvency_ratio,1.082930
4,ABB,2020.0,0.0,return_on_investment,0.009605
...,...,...,...,...,...
12031,VPB,2024.0,2.0,CFO_to_total_equity,0.119020
12032,VPB,2024.0,2.0,cash_flow_from_sales_to_sales,0.909957
12033,VPB,2024.0,2.0,cash_flow_margin,-0.000346
12034,VPB,2024.0,2.0,earning_quality_ratio,4.551894


In [45]:
ratios_df.ratio_code= ratios_df.ratio_code.str.replace('_', ' ').str.title()

In [46]:
ratios_df

,stock_code,year,quarter,ratio_code,data
0,ABB,2020.0,0.0,Equity Ratio,0.076580
1,ABB,2020.0,0.0,Long Term Asset Self Financing Ratio,7.475770
2,ABB,2020.0,0.0,Fixed Asset Self Financing Ratio,9.324157
3,ABB,2020.0,0.0,General Solvency Ratio,1.082930
4,ABB,2020.0,0.0,Return On Investment,0.009605
...,...,...,...,...,...
12031,VPB,2024.0,2.0,Cfo To Total Equity,0.119020
12032,VPB,2024.0,2.0,Cash Flow From Sales To Sales,0.909957
12033,VPB,2024.0,2.0,Cash Flow Margin,-0.000346
12034,VPB,2024.0,2.0,Earning Quality Ratio,4.551894


In [47]:
ratios_df.ratio_code.to_csv('ratios_del.csv', index=False)

In [50]:
df_rename = pd.read_csv('ratios_del.csv')
df_rename.rename(columns={'ratio_code':'Ratio'}, inplace=True)


In [51]:
ratios_df = pd.concat([ratios_df, df_rename], axis=1)
ratios_df.drop(columns=['ratio_code'], inplace=True)
ratios_df.rename(columns={'Ratio':'ratio_code'}, inplace=True)

In [52]:
ratios_df

,stock_code,year,quarter,data,ratio_code
0,ABB,2020.0,0.0,0.076580,Equity Ratio
1,ABB,2020.0,0.0,7.475770,Long Term Asset Self Financing Ratio
2,ABB,2020.0,0.0,9.324157,Fixed Asset Self Financing Ratio
3,ABB,2020.0,0.0,1.082930,General Solvency Ratio
4,ABB,2020.0,0.0,0.009605,Return On Investment
...,...,...,...,...,...
12031,VPB,2024.0,2.0,0.119020,CFO To Total Equity
12032,VPB,2024.0,2.0,0.909957,Cash Flow From Sales To Sales
12033,VPB,2024.0,2.0,-0.000346,Cash Flow Margin
12034,VPB,2024.0,2.0,4.551894,Earning Quality Ratio


In [37]:
df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\sample\sample_data\map_ratios_codes.csv')
df.rename(columns={'ratio_code': 'ratio_','ratio_name':'ratio_code'}, inplace=True)
df.head()

,ratio_code,ratio_
0,Equity Ratio,ER
1,Long Term Asset Self Financing Ratio,LTASFR
2,Fixed Asset Self Financing Ratio,FASFR
3,General Solvency Ratio,GSR
4,Return On Investment,ROI


In [53]:
merged_df = pd.merge(ratios_df,df, on='ratio_code', how='left')

# Display the merged DataFrame
merged_df

,stock_code,year,quarter,data,ratio_code,ratio_
0,ABB,2020.0,0.0,0.076580,Equity Ratio,ER
1,ABB,2020.0,0.0,7.475770,Long Term Asset Self Financing Ratio,LTASFR
2,ABB,2020.0,0.0,9.324157,Fixed Asset Self Financing Ratio,FASFR
3,ABB,2020.0,0.0,1.082930,General Solvency Ratio,GSR
4,ABB,2020.0,0.0,0.009605,Return On Investment,ROI
...,...,...,...,...,...,...
12031,VPB,2024.0,2.0,0.119020,CFO To Total Equity,CFOTTE
12032,VPB,2024.0,2.0,0.909957,Cash Flow From Sales To Sales,CFFSTS
12033,VPB,2024.0,2.0,-0.000346,Cash Flow Margin,CFM
12034,VPB,2024.0,2.0,4.551894,Earning Quality Ratio,EQR


In [54]:
merged_df.drop(columns=['ratio_code'], inplace=True)
merged_df.rename(columns={'ratio_': 'ratio_code'}, inplace=True)
merged_df

,stock_code,year,quarter,data,ratio_code
0,ABB,2020.0,0.0,0.076580,ER
1,ABB,2020.0,0.0,7.475770,LTASFR
2,ABB,2020.0,0.0,9.324157,FASFR
3,ABB,2020.0,0.0,1.082930,GSR
4,ABB,2020.0,0.0,0.009605,ROI
...,...,...,...,...,...
12031,VPB,2024.0,2.0,0.119020,CFOTTE
12032,VPB,2024.0,2.0,0.909957,CFFSTS
12033,VPB,2024.0,2.0,-0.000346,CFM
12034,VPB,2024.0,2.0,4.551894,EQR


In [55]:
merged_df.to_csv('sample/sample_data/financial_ratios_bank.csv', index=False)